<a href="https://colab.research.google.com/github/Abde-Ali/Amazon-Food-Review/blob/main/Amazon_Review_True_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/MachineLearning Projects/Reviews.csv")

In [3]:
df['Sentiment'] = df['Score'].apply(lambda x: 0 if x<3 else 1)
sorted = df.sort_values("ProductId", axis=0,ascending=True,inplace=False, kind='quicksort', na_position='last')
data = sorted.drop_duplicates(subset={"UserId","ProfileName","Time","ReviewText"},keep='first',inplace=False)
data.shape

(393758, 10)

In [4]:
#Preprocessing
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

eng_sw = stopwords.words('english')
eng_sw = [w for w in eng_sw if w not in ["not","wasn't","didn't","haven't","weren't"] ]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
def preprocessing(text):
  #text = text.lower()
  text = re.sub(r'[^A-z]', '', str(text)).lower()
  tokens = word_tokenize(text)
  tokens = [w for w in tokens if w not in eng_sw]
  tokens = [stemmer.stem(w) for w in tokens]
  return" ".join(tokens)

In [6]:
df_sample = data.sample(n=50000)
df_sample["ProccessedSummary"] = df_sample['ReviewSummary'].apply(preprocessing)
df_sample["ProccessedText"] = df_sample["ReviewText"].apply(preprocessing)

In [7]:
tfidf = TfidfVectorizer(max_features=3000)
x_num = df_sample[['HelpfulnessNumerator', 'HelpfulnessDenominator']]
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_num = sc.fit_transform(x_num)

In [8]:
x_text = tfidf.fit_transform(df_sample["ProccessedSummary"])
x_text1 = tfidf.fit_transform(df_sample["ProccessedText"])

In [9]:
data['Sentiment'] = data['Score'].apply(lambda x: 0 if x<3 else 1)
x = np.hstack([x_text.toarray(),x_text1.toarray(),x_train_num])
y = df_sample["Sentiment"]

<ipython-input-9-242779cde2ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Sentiment'] = data['Score'].apply(lambda x: 0 if x<3 else 1)


In [10]:
print(x.shape,y.shape)

(50000, 6002) (50000,)


In [11]:
x_train,x_test ,y_train, y_test = train_test_split(x,y,test_size=0.25)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
mb = MultinomialNB()
gb = GaussianNB()
dt = DecisionTreeClassifier()
lr = LogisticRegression(max_iter=1000)
rfc=RandomForestClassifier()
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix

In [15]:
lr.fit(x_train,y_train)
y_pred_lr = lr.predict(x_test)

In [16]:
gb.fit(x_train,y_train)
y_pred_gb = gb.predict(x_test)

In [18]:
dt.fit(x_train,y_train)
y_pred_dt = dt.predict(x_test)

In [19]:
rfc.fit(x_train,y_train)
y_pred_rfc = rfc.predict(x_test)

In [20]:
print(f'accurarcy Score: {accuracy_score(y_test, y_pred_lr)}\nprecision Score: {precision_score(y_test, y_pred_lr)}\nRecall Score: {recall_score(y_test, y_pred_lr)}\nF1 Score: {f1_score(y_test, y_pred_lr)}')

accurarcy Score: 0.87976
precision Score: 0.8837074463679284
Recall Score: 0.9908060921248143
F1 Score: 0.9341972768267589


In [21]:
print(f'accurarcy Score: {accuracy_score(y_test, y_pred_dt)}\nprecision Score: {precision_score(y_test, y_pred_dt)}\nRecall Score: {recall_score(y_test, y_pred_dt)}\nF1 Score: {f1_score(y_test, y_pred_dt)}')

accurarcy Score: 0.87656
precision Score: 0.8882248968942008
Recall Score: 0.9800334323922734
F1 Score: 0.9318733718927988


In [22]:
print(f'accurarcy Score: {accuracy_score(y_test, y_pred_rfc)}\nprecision Score: {precision_score(y_test, y_pred_rfc)}\nRecall Score: {recall_score(y_test, y_pred_rfc)}\nF1 Score: {f1_score(y_test, y_pred_rfc)}')

accurarcy Score: 0.88016
precision Score: 0.890875358407826
Recall Score: 0.9810549777117384
F1 Score: 0.9337929815256784


In [23]:
print(f'accurarcy Score: {accuracy_score(y_test, y_pred_gb)}\nprecision Score: {precision_score(y_test, y_pred_gb)}\nRecall Score: {recall_score(y_test, y_pred_gb)}\nF1 Score: {f1_score(y_test, y_pred_gb)}')

accurarcy Score: 0.36448
precision Score: 0.98790601243953
Recall Score: 0.26550891530460624
F1 Score: 0.41853315766359245


In [24]:
#print(f'accurarcy Score: {accuracy_score(y_test, y_pred_mb)}\nprecision Score: {precision_score(y_test, y_pred_mb)}\nRecall Score: {recall_score(y_test, y_pred_mb)}\nF1 Score: {f1_score(y_test, y_pred_mb)}')

In [25]:
#mb.fit(x_train,y_train)
#y_pred_mb = mb.predict(x_test)